In [3]:
import ccxt
import pandas as pd

In [4]:
exchange = ccxt.binance()
symbol = 'BTC/USDT'
df_bid = pd.DataFrame(exchange.fetch_order_book(symbol, 10000)['bids'], columns=['price', 'quantity'])
df_ask = pd.DataFrame(exchange.fetch_order_book(symbol, 50)['asks'], columns=['price', 'quantity'])
df_bid['usd_quantity'] = df_bid['price'] *  df_bid['quantity']
df_ask['usd_quantity'] = df_ask['price'] *  df_ask['quantity']

In [5]:
df_bid

,price,quantity,usd_quantity
0,17215.74,0.02576,443.477462
1,17215.72,0.05241,902.275885
2,17215.70,0.05241,902.274837
3,17215.58,0.00059,10.157192
4,17215.57,0.01689,290.770977
...,...,...,...
4995,16940.93,0.00060,10.164558
4996,16940.90,0.00349,59.123741
4997,16940.89,0.00062,10.503352
4998,16940.75,0.00064,10.842080


In [6]:
def get_spread_from_order_book(exchange, symbol, usd_amount, show_order=False):
    df_bid = pd.DataFrame(exchange.fetch_order_book(symbol, 5000)['bids'], columns=['price', 'quantity'])
    df_ask = pd.DataFrame(exchange.fetch_order_book(symbol, 5000)['asks'], columns=['price', 'quantity'])
    df_bid['usd_quantity'] = df_bid['price'] *  df_bid['quantity']
    df_ask['usd_quantity'] = df_ask['price'] *  df_ask['quantity']

    i = 0
 
    for j in range(0,2,1): 
        operations_array = []
        usd_spent = 0
        if j == 0:
            df = df_ask.copy()
            # print(df)
        else:
            df = df_bid.copy()  
            # print(df)
        while(usd_spent < usd_amount):
            if df.iloc[i]['usd_quantity'] > usd_amount - usd_spent:
                operations_array.append({"amount":usd_amount - usd_spent, "price":df.iloc[i]['price']})
                usd_spent += (usd_amount - usd_spent)
            else: 
                operations_array.append({"amount":df.iloc[i]['usd_quantity'], "price":df.iloc[i]['price']})
                usd_spent += df.iloc[i]['usd_quantity']
            i += 1
            
        dt = pd.DataFrame(operations_array)
        dt['sum'] = dt['amount'] * dt['price']
        mean_price = dt['sum'].sum() / dt['amount'].sum()
        mid_bid_ask_price = (df_bid.iloc[0]["price"]+df_ask.iloc[0]["price"])/2
        diff_spread = abs(mid_bid_ask_price - mean_price) / mean_price
        spread_fee = usd_amount * diff_spread
        del dt['sum']
        if j == 0:
            print("||| Mid Bid Ask Price :",mid_bid_ask_price,"$ |||")
            print("\n------------ BUY---------")
        else:
            print("\n------------ SELL---------")
        
        print("||| Mean Operation Price  =",mean_price,"$ |||")
        # print("||| Mid Bid Ask Price =", mid_bid_ask_price,"$ |||")
        print("||| Spread Differance =", round(diff_spread*100,4),"% |||")
        print("||| Spread Fees =", round(spread_fee,2),"$ |||")
        if show_order:
            print("-"*30)
            print("--- Operation array")
            print(dt.to_string(index=False))


exchange = ccxt.binance()
get_spread_from_order_book(
    exchange = exchange,
    symbol = "GRT/USDT", 
    usd_amount = 100000,
    show_order = False
)
        

||| Mid Bid Ask Price : 0.0673 |||

------------ BUY---------
||| Mean Operation Price  = 0.0675960859201 |||
||| Spread Differance = 0.438 % |||
||| Spread Fees = 438.02 |||

------------ SELL---------
||| Mean Operation Price  = 0.06527985035910001 |||
||| Spread Differance = 3.0946 % |||
||| Spread Fees = 3094.6 |||
